In [68]:
import math

def get_roll(face_landmarks):
    left_eye_center_x = (face_landmarks[133].x - face_landmarks[33].x)/2
    left_eye_center_y = (face_landmarks[133].y - face_landmarks[33].y)/2
    right_eye_center_x = (face_landmarks[362].x - face_landmarks[263].x)/2
    right_eye_center_y = (face_landmarks[362].y - face_landmarks[263].y)/2
    dx = abs(right_eye_center_x - left_eye_center_x)
    dy = abs(right_eye_center_y - left_eye_center_y)

    # Calculate roll in degrees
    roll = abs(math.atan2(dy, dx)) / math.pi * 180
    return roll

In [69]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

# STEP 2: Create an FaceLandmarker object.
def get_landmarks(image_path):
    base_options = python.BaseOptions(model_asset_path='../utilities/face_landmarker.task')
    options = vision.FaceLandmarkerOptions(base_options=base_options,
                                        output_face_blendshapes=True,
                                        output_facial_transformation_matrixes=True,
                                        num_faces=1)
    detector = vision.FaceLandmarker.create_from_options(options)

    # STEP 3: Load the input image
    frame = cv2.imread(image_path)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)

    # STEP 5: Process the detection result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

    face_landmarks_list = detection_result.face_landmarks
    face_landmarks = face_landmarks_list[0]
    return face_landmarks

In [70]:
import glob
import pandas as pd
from datetime import datetime

all_images = glob.glob("D:/oneid_backend/icao-guidelines-server/dataset/*.JPG")

In [71]:
def logger(image, is_valid, path="./"):
    time_str = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_headers = pd.DataFrame({
        'log_time': [time_str],
        'image': [image],
        'is_valid': [is_valid],
    })
    df_headers.to_csv(f'{path}roll_pitch_yaw_test.csv', mode='a', index=False, header=False)

In [72]:
# time duration
start_time = datetime.now()

for image_path in all_images:
    landmarks = get_landmarks(image_path)
    roll = get_roll(landmarks)
    logger(image_path, roll)
    print(image_path, roll)

D:/oneid_backend/icao-guidelines-server/dataset\N230101660.JPG 0.5651738121595722
D:/oneid_backend/icao-guidelines-server/dataset\N230101661.JPG 0.6084917436975289
D:/oneid_backend/icao-guidelines-server/dataset\N230101664.JPG 2.195167524891388
D:/oneid_backend/icao-guidelines-server/dataset\N230101669.JPG 1.9947938219416537
D:/oneid_backend/icao-guidelines-server/dataset\N230101671.JPG 0.44064438367315434
D:/oneid_backend/icao-guidelines-server/dataset\N230101683.JPG 0.7718539318221354
D:/oneid_backend/icao-guidelines-server/dataset\N230101691.JPG 0.03096047847218954
D:/oneid_backend/icao-guidelines-server/dataset\N230101696.JPG 0.05398599778694913
D:/oneid_backend/icao-guidelines-server/dataset\N230101709.JPG 1.102633110407097
D:/oneid_backend/icao-guidelines-server/dataset\N230101710.JPG 0.021520819054318775
D:/oneid_backend/icao-guidelines-server/dataset\N230101717.JPG 0.36456943933637576
D:/oneid_backend/icao-guidelines-server/dataset\N230101719.JPG 3.533660314128211
D:/oneid_back